In [1]:
!pip install folium

In [2]:
!pip install scikit-learn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import folium
import numpy as np
from folium.plugins import HeatMap
from sklearn.cluster import KMeans
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

Load-in Data

In [5]:
# Note you may need to change the filename or how you load it in to replicate
file_name = '/content/drive/MyDrive/2023_Walk___Bike_Count_Data_20250721.csv'

df = pd.read_csv(file_name)

relevant_cols = ['Street Name', 'Limit 1', 'Limit 2', 'Ped_Total', 'Bike_Total', 'Scooter_Total', 'Latitude', 'Longitude']
df = df[relevant_cols]

df['Total_Activity'] = df[['Ped_Total', 'Bike_Total', 'Scooter_Total']].fillna(0).sum(axis=1)

df = df.dropna(subset=['Latitude', 'Longitude', 'Total_Activity'])

Heatmap code

In [6]:
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
base_map = folium.Map(location=map_center, zoom_start=12)

heat_data = df[['Latitude', 'Longitude', 'Total_Activity']].values.tolist()
HeatMap(heat_data, radius=15, max_zoom=13).add_to(base_map)

output_path = '/content/drive/MyDrive/la_bike_ped_heatmap.html'
base_map.save(output_path)

K Means with Naive BRT connections (full graph)

In [17]:
k = 6  # you can change this depending on how many BRT zones you want
kmeans = KMeans(n_clusters=k, random_state=0)
df['cluster'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

centroids = kmeans.cluster_centers_

map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
base_map = folium.Map(location=map_center, zoom_start=12)

# Add clustered points to map
marker_cluster = MarkerCluster().add_to(base_map)
for _, row in df.iterrows():
    folium.CircleMarker(
        location=(row['Latitude'], row['Longitude']),
        radius=3,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(marker_cluster)

# Add cluster centers (naive BRT stops)
for i, (lat, lon) in enumerate(centroids):
    folium.Marker(
        location=(lat, lon),
        popup=f"BRT Stop {i+1}",
        icon=folium.Icon(color='red', icon='bus', prefix='fa')
    ).add_to(base_map)

# Draw naive BRT corridors by connecting cluster centers
for i in range(len(centroids)):
    for j in range(i + 1, len(centroids)):
        folium.PolyLine(
            locations=[tuple(centroids[i]), tuple(centroids[j])],
            color='orange',
            weight=4,
            opacity=1
        ).add_to(base_map)

output_path = '/content/drive/MyDrive/kmeans_naive_brt_map.html'
base_map.save(output_path)

K Means with MST BRT connections

In [6]:
k = 8  # you can change this depending on how many BRT zones you want
kmeans = KMeans(n_clusters=k, random_state=0)
df['cluster'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

centroids = kmeans.cluster_centers_

map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
base_map = folium.Map(location=map_center, zoom_start=12)

# Add clustered points to map
marker_cluster = MarkerCluster().add_to(base_map)
for _, row in df.iterrows():
    folium.CircleMarker(
        location=(row['Latitude'], row['Longitude']),
        radius=3,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(marker_cluster)

# Add cluster centers (naive BRT stops)
for i, (lat, lon) in enumerate(centroids):
    folium.Marker(
        location=(lat, lon),
        popup=f"BRT Stop {i+1}",
        icon=folium.Icon(color='red', icon='bus', prefix='fa')
    ).add_to(base_map)

centroid_coords = [tuple(c) for c in centroids]
k = len(centroid_coords)

# Build a full distance matrix
dist_matrix = [[0]*k for _ in range(k)]
for i in range(k):
    for j in range(k):
        if i != j:
            dist_matrix[i][j] = geodesic(centroid_coords[i], centroid_coords[j]).meters
        else:
            dist_matrix[i][j] = float('inf')  # no self-loops

visited = set()
edges = []

visited.add(0)

while len(visited) < k:
    min_edge = (None, None, float('inf'))  # (i, j, dist)
    for i in visited:
        for j in range(k):
            if j not in visited and dist_matrix[i][j] < min_edge[2]:
                min_edge = (i, j, dist_matrix[i][j])
    i, j, _ = min_edge
    visited.add(j)
    edges.append((i, j))

# Draw MST edges
for i, j in edges:
    folium.PolyLine(
        locations=[centroid_coords[i], centroid_coords[j]],
        color='orange',
        weight=3,
        opacity=0.8
    ).add_to(base_map)

output_path = '/content/drive/MyDrive/kmeans_MST_brt_map.html'
base_map.save(output_path)